In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error


url = 'https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/demand_data_exercise.csv'
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')


print("First few rows of the dataset:")
print(data.head())


print("\nDescriptive Statistics for 'Demand_with_Trend':")
print(data['Demand_with_Trend'].describe())


plt.figure(figsize=(12, 6))
plt.plot(data.index, data['Demand_with_Trend'], label='Demand_with_Trend')
plt.title('Demand with Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


plt.figure(figsize=(10, 5))
plt.hist(data['Demand_with_Trend'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Demand with Trend')
plt.xlabel('Demand')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]


train['EMA'] = train['Demand_with_Trend'].ewm(span=10, adjust=False).mean()


last_train_ema = train['EMA'].iloc[-1]
test['EMA'] = last_train_ema


plt.figure(figsize=(12, 6))
plt.plot(train.index, train['Demand_with_Trend'], label='Training Data - Demand_with_Trend')
plt.plot(train.index, train['EMA'], label='Training EMA', color='red')
plt.plot(test.index, test['Demand_with_Trend'], label='Testing Data - Demand_with_Trend')
plt.plot(test.index, test['EMA'], label='Testing EMA (Constant)', color='red', linestyle='--')
plt.title('Exponential Moving Average (EMA) Applied Only on Training Data')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


arima_model = ARIMA(train['Demand_with_Trend'], order=(5, 1, 0)).fit()


train['ARIMA_Forecast'] = arima_model.fittedvalues
test['ARIMA_Forecast'] = arima_model.forecast(steps=len(test))


plt.figure(figsize=(12, 6))
plt.plot(train.index, train['Demand_with_Trend'], label='Training Data')
plt.plot(test.index, test['Demand_with_Trend'], label='Test Data')
plt.plot(test.index, test['ARIMA_Forecast'], label='ARIMA Forecast', color='green')
plt.title('ARIMA Model - Actual vs. Forecasted Demand_with_Trend')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


mae_ema = mean_absolute_error(test['Demand_with_Trend'], test['EMA'])
mape_ema = np.mean(np.abs((test['Demand_with_Trend'] - test['EMA']) / test['Demand_with_Trend'])) * 100
rmse_ema = np.sqrt(mean_squared_error(test['Demand_with_Trend'], test['EMA']))

mae_arima = mean_absolute_error(test['Demand_with_Trend'], test['ARIMA_Forecast'])
mape_arima = np.mean(np.abs((test['Demand_with_Trend'] - test['ARIMA_Forecast']) / test['Demand_with_Trend'])) * 100
rmse_arima = np.sqrt(mean_squared_error(test['Demand_with_Trend'], test['ARIMA_Forecast']))


print("Evaluation Metrics for EMA (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_ema:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_ema:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_ema:.2f}")

print("\nEvaluation Metrics for ARIMA (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_arima:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_arima:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_arima:.2f}")
